In [ ]:
using CSV, Tables
using JuMP
using Gurobi

import Pkg; Pkg.add("JSON3")
using JSON3

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [4]:
const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-08


## load in data

In [5]:
all_classes = open("data/all_classes_info.json", "r") do file 
    JSON3.read(file)
end

R = CSV.File("data/ratings.csv",header=0) |> Tables.matrix;
H = CSV.File("data/hours.csv",header=0) |> Tables.matrix;
U = CSV.File("data/units.csv",header=0) |> Tables.matrix;

vars = CSV.File("data/variables.csv",header=0) |> Tables.matrix;
n = vars[1];
S = vars[2];

In [6]:
all_classes[1]

JSON3.Object{Base.CodeUnits{UInt8, String}, SubArray{UInt64, 1, Vector{UInt64}, Tuple{UnitRange{Int64}}, true}} with 28 entries:
  :rating              => 5.4
  :gir_attribute       => "REST"
  :has_final           => false
  :description         => "Presents engineering problems in a computational set…
  :offered_fall        => false
  :offered_spring      => true
  :meets_with_subjects => ["1.001"]
  :instructors         => ["J. Williams"]
  :out_of_class_hours  => 7.99
  :total_units         => 12
  :related_subjects    => ["2.156", "1.205", "1.C51", "1.000", "1.631", "1.063"…
  :pdf_option          => false
  :in_class_hours      => 5.7
  :is_half_class       => false
  :level               => "U"
  :prerequisites       => "GIR:CAL1"
  :subject_id          => "1.00"
  :title               => "Engineering Computation and Data Science"
  :lab_units           => 2
  :design_units        => 0
  :public              => true
  :offered_summer      => false
  :lecture_units       => 3
  :

In [8]:
model = Model(() -> Gurobi.Optimizer(GRB_ENV))

α = 0.8

# @variable(model, X[1:n,1:s] >=0)
@variable(model, X[1:n, 1:S], Bin)

@objective(model, Min, sum(α*H[i]*X[i,s] - (1-α)*R[i]*X[i,s] for i=1:n,s=1:S));

# full-time; at least 36 units per semester
@constraint(model, [s in 1:S], sum(U[i]*X[i, s] for i=1:n) >= 36);
# wellbeing: no more than 112 hours
@constraint(model, [s in 1:S], sum(H[i]*X[i, s] for i=1:n) <= 112);
# not repeatable: cannot take a class again
for i in 1:n
    for t in 1:(S - 1)
        @constraint(model, sum(X[i, t:end]) <= 1)
    end
end

optimize!(model)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 32937 rows, 37624 columns and 235589 nonzeros
Model fingerprint: 0x6dee8709
Variable types: 0 continuous, 37624 integer (37624 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [2e-02, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 158.2820000
Presolve removed 28749 rows and 4248 columns
Presolve time: 0.76s
Presolved: 4188 rows, 33376 columns, 100112 nonzeros
Variable types: 0 continuous, 33376 integer (33376 binary)
Found heuristic solution: objective 119.7880000
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Use crossover to convert LP symmetric solution to

In [9]:
taken_classes_sem = [[ind for (ind, val) in enumerate(sem) if val == 1] for sem in eachcol(value.(X))]

8-element Vector{Vector{Int64}}:
 [1953, 2021, 3984, 4141]
 [294, 296, 974, 2885, 2892, 3017, 3994]
 [1914, 2625, 2915, 2919, 4492, 4557]
 [144, 714, 3369, 4347]
 [6, 978, 1025, 1666, 1946]
 [816, 1324, 2866, 3149]
 [725, 1180, 2600, 3337, 4160]
 [100, 1350, 4220, 4511]

## Interpreter

In [10]:
function interpret_course_nums(classes_id_per_sem)
    course_nums = [[all_classes[class_ind][:subject_id] for class_ind in sem] for sem in classes_id_per_sem]
    return course_nums
end

interpret_course_nums (generic function with 1 method)

In [11]:
interpret_course_nums(taken_classes_sem)

8-element Vector{Vector{String}}:
 ["20.053", "20.S900", "9.271", "CMS.627"]
 ["10.960", "10.991", "15.323", "24.93", "24.947", "3.903", "9.357"]
 ["2.991", "21M.747", "3.001", "3.006", "MAS.921", "SP.257"]
 ["1.713", "12.834", "6.072", "HST.533"]
 ["1.008", "15.335", "15.3941", "18.727", "20.001"]
 ["14.399", "16.634", "24.805", "4.359"]
 ["12.900", "15.839", "21M.622", "5.561", "CMS.827"]
 ["1.267", "16.84", "EC.110", "MS.101"]